In [25]:
%pip install -q -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain import hub
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
def print_output(docs):
    for doc in docs:
        print('The output is: {}. \n\nThe metadata is {} \n\n'.
              format(doc.page_content, doc.metadata))    

In [4]:
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

In [5]:
db = FAISS.load_local("summarizer_index", 
                      OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small"), 
                      allow_dangerous_deserialization=True)

In [6]:
question = """
Given the development and individual item reports and the LinkedIn profile description,
generate a short profile summary of this person, highlighting the achievements
and skills.
Use generic terms instead of specific examples.
Apply skills categories when possible.
"""
docs = db.similarity_search(question)

In [7]:
print_output(docs)

The output is: My Development Items
 02:35 PM
11/18/2024
Page 1 of 5
Daniele Martinoli
Development Item
Effective communication
Additional Information
Creating comprehensible and appealing presentations.
Understanding the real needs of the partners and being able to translate it to the recommended Red Hat architecture and solution.
Skills to Develop
Relates To
Status
In Progress
Start Date
Completion Date
12/31/2023
Status Note
Picked some useful video trainings from LinkedIn Learning to start this task 
July 23: Started listening "Amplifying Your Message Through Powerful Framing and Storytelling" podcast on LinkeId Learnings 
https://www.linkedin.com/learning/amplifying-your-message-through-powerful-framing-and-storytelling 
------------ 
July 25, 2023 update 
It was not a meaningful learning, more dedicated to people who want to sell their ideas, and it's not me. 
Have to look for something different.
Development Item
Learning AI
Additional Information. 

The metadata is {'source': '

In [8]:
# initialize the LLM we'll use - OpenAI GPT 3.5 Turbo
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")

In [9]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [10]:
prompt = hub.pull("rlm/rag-prompt")

/Users/dmartino/projects/AI/learning/chat-with-your-data-using-chatgpt-3806110/venv/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [12]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# combine multiple steps in a single chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()  # convert the chat message to a string
)

In [23]:
question = """
You are the person for which we provide the development and individual item reports
and the LinkedIn profile description.
Generate a profile description for yourself.
The profile must include:
- one paragraph with the description of professional role, highlighting the competences
- one paragraph with a comma separated list of recent achievements
- one paragraph with a comma separated list of skills

Use first person as "I am" instead of "He is".
Use generic terms instead of specific examples: "DevOps" instead of "Jenkins".
Avoid to mention customer names like Temenos.
Use skills categorizies when possible.
At the end of each sentence add a list of the reference source files.
At he end of the response, include a count of words.
"""

In [24]:
for chunk in rag_chain.stream(
    question
):
    print(chunk, end="", flush=True)

I am a Principal Software Engineer with a proven track record in software architecture, design, and development, specializing in Java, Spring, Hibernate, and Agile methodologies (Scrum). I excel in problem-solving with an analytical mindset and have a strong educational background, always eager to learn and apply new concepts efficiently (LinkedIn profile description, Resume).

Recent achievements include inspiring a 200+ member development team, earning the highest technical grade in the company, and constantly delivering top-ranking performance evaluations (Reference source: Resume). Skills include Software Architecture, TDD, MDD, Java, Spring, Hibernate, REST, and Agile methodologies (Scrum) (LinkedIn profile description, Resume).

My skills encompass Software Architecture, Design and Development, Test-Driven Development (TDD), Model-Driven Development (MDD), Java, Spring, Hibernate, REST, and Agile methodologies (Scrum) (LinkedIn profile description, Resume). (Word count: 98)